# Movimientos

Este notebook demuestra una interfaz mínima de "Movimientos" usando Tkinter.
Se crea una pestaña/ventana llamada "Movimientos" con un frame de filtros y una tabla (ttk.Treeview).


## Objetivo

Crear una vista llamada "Movimientos" que contenga:
- Un frame de filtros (Tipo / Desde / Hasta / Aplicar Filtros)
- Una tabla (Treeview) con las columnas: Nro, Fecha, Tipo, Producto, Cantidad, Ubicación, Responsable, Referencia

El ejemplo usa un hilo para ejecutar la GUI sin bloquear el notebook y datos de ejemplo para demo.

In [ ]:
# Section 1: Import Required Libraries
import tkinter as tk
from tkinter import ttk, messagebox
import threading
from datetime import datetime
from IPython.display import display
import pandas as pd

# Small aliases for typing clarity
ROOT = None
GUI_THREAD = None


In [ ]:
# Section 2: Start Tkinter in Notebook (threaded)

def _run_gui(start_fn):
    """Internal helper that runs the provided start_fn in the current thread."""
    try:
        start_fn()
    except Exception as e:
        print("GUI thread error:", e)


def start_gui():
    """Start the Tkinter GUI on a daemon thread so the notebook stays responsive."""
    global ROOT, GUI_THREAD

    if GUI_THREAD and GUI_THREAD.is_alive():
        print("GUI ya está ejecutándose")
        return

    def _target():
        global ROOT
        ROOT = tk.Tk()
        build_ui(ROOT)
        ROOT.mainloop()

    GUI_THREAD = threading.Thread(target=_target, daemon=True)
    GUI_THREAD.start()
    print("GUI iniciada en hilo de fondo")


def stop_gui():
    """Stop the GUI if running."""
    global ROOT
    if ROOT:
        try:
            ROOT.quit()
            ROOT.destroy()
        except Exception as e:
            print("Error cerrando GUI:", e)



In [ ]:
# Section 3: Create main 'Movimientos' frame and title

def build_ui(root):
    """Construct the Movimientos UI inside the provided root window."""
    # Basic window settings
    root.title("Movimientos - Demo")
    root.geometry("1000x600")

    # Main container
    main_frame = ttk.Frame(root, padding=10)
    main_frame.pack(fill="both", expand=True)

    # Title
    title_label = ttk.Label(main_frame, text="Movimientos", font=("Segoe UI", 14, "bold"))
    title_label.pack(anchor="w", pady=(0, 10))

    # We'll create filter_frame and table_frame below
    # Attach to main_frame so they share the same parent
    root._mov_main_frame = main_frame



In [ ]:
# Section 4: Build Filters frame (Tipo, Desde, Hasta, Aplicar)

def build_filters(main_frame):
    filter_frame = ttk.Frame(main_frame)
    filter_frame.pack(fill="x", pady=(0, 10))

    # Tipo
    ttk.Label(filter_frame, text="Tipo:").pack(side="left", padx=5)
    tipo_combo = ttk.Combobox(filter_frame, values=["Todos", "Entrada", "Salida"], state="readonly", width=12)
    tipo_combo.set("Todos")
    tipo_combo.pack(side="left", padx=5)

    # Desde
    ttk.Label(filter_frame, text="Desde (YYYY-MM-DD):").pack(side="left", padx=5)
    desde_entry = ttk.Entry(filter_frame, width=12)
    desde_entry.pack(side="left", padx=5)

    # Hasta
    ttk.Label(filter_frame, text="Hasta (YYYY-MM-DD):").pack(side="left", padx=5)
    hasta_entry = ttk.Entry(filter_frame, width=12)
    hasta_entry.pack(side="left", padx=5)

    # Aplicar
    apply_btn = ttk.Button(filter_frame, text="Aplicar Filtros")
    apply_btn.pack(side="left", padx=10)

    return {
        'frame': filter_frame,
        'tipo_combo': tipo_combo,
        'desde_entry': desde_entry,
        'hasta_entry': hasta_entry,
        'apply_btn': apply_btn
    }



In [ ]:
# Section 5: Create Movements Treeview and Scrollbar

def build_table(main_frame):
    table_frame = ttk.Frame(main_frame)
    table_frame.pack(fill="both", expand=True)

    columns = ("Nro", "Fecha", "Tipo", "Producto", "Cantidad", "Ubicación", "Responsable", "Referencia")
    tree = ttk.Treeview(table_frame, columns=columns, show="headings", height=15)

    col_widths = [50, 140, 80, 180, 80, 120, 120, 160]
    for col, w in zip(columns, col_widths):
        tree.heading(col, text=col)
        tree.column(col, width=w)

    vsb = ttk.Scrollbar(table_frame, orient="vertical", command=tree.yview)
    tree.configure(yscrollcommand=vsb.set)

    tree.pack(side="left", fill="both", expand=True)
    vsb.pack(side="right", fill="y")

    return tree


def populate_table(tree, data):
    tree.delete(*tree.get_children())
    for i, row in enumerate(data, start=1):
        tree.insert("", "end", values=(i,)+row)



In [ ]:
# Section 6: Controller & Mock Data: refresh_movements_table

# Mock dataset: tuples of (fecha_str, tipo, producto, cantidad, ubicacion, responsable, referencia)
MOCK_MOVEMENTS = [
    ("2025-10-01", "Entrada", "Toner HP", 10, "Almacén A", "Ana", "Pedido #123"),
    ("2025-10-05", "Salida", "Monitor 22\"", 2, "Tienda", "Carlos", "Venta"),
    ("2025-10-10", "Entrada", "Teclado", 5, "Almacén B", "Lucía", "Devolución"),
    ("2025-10-20", "Salida", "Cables HDMI", 3, "Almacén A", "Miguel", "Reemplazo")
]


def parse_date_safe(s):
    if not s:
        return None
    try:
        return datetime.strptime(s.strip(), "%Y-%m-%d")
    except Exception:
        return None


def refresh_movements_table(tree, movement_type, date_from, date_to):
    df = pd.DataFrame(MOCK_MOVEMENTS, columns=["fecha", "tipo", "producto", "cantidad", "ubicacion", "responsable", "referencia"]) 
    # Parse fecha
    df["fecha_dt"] = pd.to_datetime(df["fecha"], format="%Y-%m-%d", errors="coerce")

    if movement_type and movement_type != "Todos":
        df = df[df["tipo"] == movement_type]

    df_from = parse_date_safe(date_from)
    df_to = parse_date_safe(date_to)

    if df_from:
        df = df[df["fecha_dt"] >= df_from]
    if df_to:
        df = df[df["fecha_dt"] <= df_to]

    rows = [tuple(r[1:8]) for r in df.itertuples()]  # remove index; return tuples matching columns except Nro
    populate_table(tree, rows)



In [ ]:
# Section 7: Event handlers: apply_filters and row selection

def apply_filters_handler(tree, tipo_combo, desde_entry, hasta_entry):
    tipo = tipo_combo.get()
    desde = desde_entry.get()
    hasta = hasta_entry.get()
    try:
        refresh_movements_table(tree, tipo, desde, hasta)
    except Exception as e:
        messagebox.showerror("Error", f"Error aplicando filtros: {e}")


def on_row_select(event):
    tree = event.widget
    sel = tree.selection()
    if not sel:
        return
    item = tree.item(sel[0])
    vals = item.get('values', [])
    messagebox.showinfo("Detalle movimiento", f"Fila seleccionada:\n{vals}")



In [ ]:
# Section 8: Styling (ttk.Style) and helper to assemble view

def apply_common_styles():
    style = ttk.Style()
    try:
        style.configure('Treeview', rowheight=22, font=("Segoe UI", 10))
        style.configure('Treeview.Heading', font=("Segoe UI", 10, 'bold'))
    except Exception:
        pass


def assemble_movimientos_view(root):
    main_frame = getattr(root, '_mov_main_frame', None)
    if not main_frame:
        main_frame = ttk.Frame(root, padding=10)
        main_frame.pack(fill="both", expand=True)

    # Filters
    filters = build_filters(main_frame)

    # Table
    tree = build_table(main_frame)

    # Wire events
    filters['apply_btn'].configure(command=lambda: apply_filters_handler(tree, filters['tipo_combo'], filters['desde_entry'], filters['hasta_entry']))
    tree.bind('<<TreeviewSelect>>', on_row_select)

    # Initial load
    refresh_movements_table(tree, "Todos", "", "")

    return {
        'main_frame': main_frame,
        'filters': filters,
        'tree': tree
    }

# Hook into build_ui so build_ui creates filters and table
def build_ui(root):
    # reuse previously defined build_ui signature
    try:
        # recreate main frame and title
        root.title("Movimientos - Demo")
        root.geometry("1000x600")

        main_frame = ttk.Frame(root, padding=10)
        main_frame.pack(fill="both", expand=True)
        title_label = ttk.Label(main_frame, text="Movimientos", font=("Segoe UI", 14, "bold"))
        title_label.pack(anchor="w", pady=(0, 10))

        apply_common_styles()
        assembled = assemble_movimientos_view(root)
        # save references
        root._mov_widgets = assembled
    except Exception as e:
        print("Error building UI:", e)


# Section 9: Run and stop GUI from notebook

# Call start_gui() to open the Movimientos window in a background thread.
# Use stop_gui() to close it safely.

# Example usage:
# start_gui()
# stop_gui()

print("Ejecute start_gui() en la siguiente celda para levantar la interfaz.")


In [ ]:
# Section 10: Optional: Export / Load CSV data

def load_movements_from_csv(path):
    global MOCK_MOVEMENTS
    df = pd.read_csv(path)
    # Expect columns: fecha, tipo, producto, cantidad, ubicacion, responsable, referencia
    MOCK_MOVEMENTS = [tuple(x) for x in df.to_records(index=False)]


def export_current_view_to_csv(tree, path):
    # collect visible rows
    rows = [tree.item(i)['values'][1:] for i in tree.get_children()]
    df = pd.DataFrame(rows, columns=["fecha","tipo","producto","cantidad","ubicacion","responsable","referencia"])
    df.to_csv(path, index=False)
    print(f"Exportado a {path}")



## Cómo ejecutar (Windows / PowerShell)

1. Abrir PowerShell y lanzar Jupyter Notebook o Jupyter Lab:

```powershell
# desde la carpeta del proyecto
cd "c:\Users\BrenierComputer123\Desktop\DefiniticoSC"
jupyter notebook
# o
jupyter lab
```

2. Abrir `movimientos.ipynb` en el navegador.
3. Ejecutar las celdas en orden. Cuando llegues a la celda final, ejecutar `start_gui()` abrirá la ventana de Tkinter en una ventana separada.
4. Para cerrar la ventana: ejecutar `stop_gui()`.

Nota: el GUI de Tkinter abre en una ventana nativa separada; el notebook permanece interactivo gracias al hilo en background.
